# LABELLING

In [298]:
import pandas as pd
import re
import string

In [299]:
def load_data():
    data = pd.read_csv('remove_http -with-label-dataset.csv')
    return data

In [300]:
tweet_df = load_data()
#tweet_df.head(n=50)

In [301]:
tweet_df

remove_http    Label
0    adek gue positif covid-. gejalanya pilek  batu...   Netral
1    Adi menambahkan  selain RS Rujukan Covid- dan ...   Netral
2    Afrika Selatan telah menyetujui vaksin sinovac...   Netral
3    Akibat paranoid: Ada yg bilang minum  kaleng s...   Netral
4    Alhamdulilah  orang di Jalan Pt Ibrahim telah ...  Positif
..                                                 ...      ...
371  Yg cuma bs goreng isiu    Ngamuk karna Pemberl...  Negatif
372  Sedangkan yg ini👇 habib mustofa al haddar kali...  Negatif
373     Yg mau razia siapa? Yaqut itu org NU yg ter...  Negatif
374  Apa yg terjadi saat ini  adalah wujud kegagala...  Negatif
375  Kepada yth Presiden RI . Tolong stop penerbang...  Negatif

[376 rows x 2 columns]

In [302]:
df = pd.DataFrame(tweet_df[['Label', 'remove_http']])

In [303]:
df

Label                                        remove_http
0     Netral  adek gue positif covid-. gejalanya pilek  batu...
1     Netral  Adi menambahkan  selain RS Rujukan Covid- dan ...
2     Netral  Afrika Selatan telah menyetujui vaksin sinovac...
3     Netral  Akibat paranoid: Ada yg bilang minum  kaleng s...
4    Positif  Alhamdulilah  orang di Jalan Pt Ibrahim telah ...
..       ...                                                ...
371  Negatif  Yg cuma bs goreng isiu    Ngamuk karna Pemberl...
372  Negatif  Sedangkan yg ini👇 habib mustofa al haddar kali...
373  Negatif     Yg mau razia siapa? Yaqut itu org NU yg ter...
374  Negatif  Apa yg terjadi saat ini  adalah wujud kegagala...
375  Negatif  Kepada yth Presiden RI . Tolong stop penerbang...

[376 rows x 2 columns]

In [304]:
#df.to_csv('TweetMark_Label.csv',encoding='utf8', index=False)

# CLEANING TEXT LABEL

In [305]:
#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean

In [306]:
df['tweet_clean'] = df['remove_http'].apply(lambda x: clean_tweets(x))

In [307]:
df

Label                                        remove_http  \
0     Netral  adek gue positif covid-. gejalanya pilek  batu...   
1     Netral  Adi menambahkan  selain RS Rujukan Covid- dan ...   
2     Netral  Afrika Selatan telah menyetujui vaksin sinovac...   
3     Netral  Akibat paranoid: Ada yg bilang minum  kaleng s...   
4    Positif  Alhamdulilah  orang di Jalan Pt Ibrahim telah ...   
..       ...                                                ...   
371  Negatif  Yg cuma bs goreng isiu    Ngamuk karna Pemberl...   
372  Negatif  Sedangkan yg ini👇 habib mustofa al haddar kali...   
373  Negatif     Yg mau razia siapa? Yaqut itu org NU yg ter...   
374  Negatif  Apa yg terjadi saat ini  adalah wujud kegagala...   
375  Negatif  Kepada yth Presiden RI . Tolong stop penerbang...   

                                           tweet_clean  
0    [adek, gue, positif, covid, gejala, pilek, bat...  
1    [adi, rs, rujuk, covid, produsen, oksigen, pln...  
2    [afrika, selatan, tuju, vaksin, sinovac, china...  
3    [akibat, paranoid, yg, bilang, minum, kaleng, ...  
4    [alhamdulilah, orang, jalan, pt, ibrahim, daft...  
..                                                 ...  
371  [yg, bs, goreng, isiu, ngamuk, karna, laku, pp...  
372  [yg, , habib, mustofa, al, haddar, kalimantan,...  
373  [yg, razia, yaqut, org, nu, yg, kenal, toa, ma...  
374  [yg, wujud, gagal, perintah, lindung, rakyat, ...  
375  [yth, presiden, ri, tolong, stop, terbang, man...  

[376 rows x 3 columns]

In [308]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text

In [309]:
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))

In [310]:
df

Label                                        remove_http  \
0     Netral  adek gue positif covid-. gejalanya pilek  batu...   
1     Netral  Adi menambahkan  selain RS Rujukan Covid- dan ...   
2     Netral  Afrika Selatan telah menyetujui vaksin sinovac...   
3     Netral  Akibat paranoid: Ada yg bilang minum  kaleng s...   
4    Positif  Alhamdulilah  orang di Jalan Pt Ibrahim telah ...   
..       ...                                                ...   
371  Negatif  Yg cuma bs goreng isiu    Ngamuk karna Pemberl...   
372  Negatif  Sedangkan yg ini👇 habib mustofa al haddar kali...   
373  Negatif     Yg mau razia siapa? Yaqut itu org NU yg ter...   
374  Negatif  Apa yg terjadi saat ini  adalah wujud kegagala...   
375  Negatif  Kepada yth Presiden RI . Tolong stop penerbang...   

                                           tweet_clean  \
0    [adek, gue, positif, covid, gejala, pilek, bat...   
1    [adi, rs, rujuk, covid, produsen, oksigen, pln...   
2    [afrika, selatan, tuju, vaksin, sinovac, china...   
3    [akibat, paranoid, yg, bilang, minum, kaleng, ...   
4    [alhamdulilah, orang, jalan, pt, ibrahim, daft...   
..                                                 ...   
371  [yg, bs, goreng, isiu, ngamuk, karna, laku, pp...   
372  [yg, , habib, mustofa, al, haddar, kalimantan,...   
373  [yg, razia, yaqut, org, nu, yg, kenal, toa, ma...   
374  [yg, wujud, gagal, perintah, lindung, rakyat, ...   
375  [yth, presiden, ri, tolong, stop, terbang, man...   

                                                 Tweet  
0    adek gue positif covid gejala pilek batuk dema...  
1    adi rs rujuk covid produsen oksigen pln siaga ...  
2    afrika selatan tuju vaksin sinovac china lawan...  
3    akibat paranoid yg bilang minum kaleng susu be...  
4    alhamdulilah orang jalan pt ibrahim daftar vak...  
..                                                 ...  
371  yg bs goreng isiu ngamuk karna laku ppkm darur...  
372  yg habib mustofa al haddar kalimantan pes waki...  
373  yg razia yaqut org nu yg kenal toa masjid seab...  
374  yg wujud gagal perintah lindung rakyat gagal t...  
375  yth presiden ri tolong stop terbang manusia ne...  

[376 rows x 4 columns]

In [312]:
#ambil data Tweet(bersih) - make code as a comment if don't use
df.drop(df.columns[[1,2]], axis = 1, inplace = True)

In [314]:
df

Label                                              Tweet
0     Netral  adek gue positif covid gejala pilek batuk dema...
1     Netral  adi rs rujuk covid produsen oksigen pln siaga ...
2     Netral  afrika selatan tuju vaksin sinovac china lawan...
3     Netral  akibat paranoid yg bilang minum kaleng susu be...
4    Positif  alhamdulilah orang jalan pt ibrahim daftar vak...
..       ...                                                ...
371  Negatif  yg bs goreng isiu ngamuk karna laku ppkm darur...
372  Negatif  yg habib mustofa al haddar kalimantan pes waki...
373  Negatif  yg razia yaqut org nu yg kenal toa masjid seab...
374  Negatif  yg wujud gagal perintah lindung rakyat gagal t...
375  Negatif  yth presiden ri tolong stop terbang manusia ne...

[376 rows x 2 columns]

In [315]:
df.to_csv('TweetClean_Label.csv',encoding='utf8', index=False)